# Gewählte xlsl Datei in csv Datei umwandeln und umbenennen

In [ ]:
# Schritt 1: Deinstalliere alle problematischen Pakete
!pip uninstall tensorflow tensorflow-text tf-keras tensorflow-decision-forests tensorflow-probability -y

# Schritt 2: Installiere kompatible Versionen
!pip install tensorflow==2.19.0 tensorflow-probability==0.25.0 tensorflow-text==2.19.0 tf-keras==2.19.0

# Schritt 3: Unterdrücke Warnungen (für parakeet und CUDA)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Unterdrückt TensorFlow-Warnungen

# Schritt 4: Teste den Import
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
print("Import erfolgreich!")

# Schritt 5: Teste tensorflow_probability
import tensorflow_probability as tfp
print(f"TensorFlow Probability Version: {tfp.__version__}")

In [1]:
import platform
import os
import pandas as pd
import random
from tqdm import tqdm
import tensorflow_probability as tfp
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from google.colab import files, drive

#from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datetime import datetime
import shutil



In [2]:
def smartEncoding():
  plt = platform.system
  if plt == "Windows":
    return "utf-8-sig"
  else:
    return "utf-8"

In [ ]:

read_file = pd.DataFrame(pd.read_excel('/content/drive/My Drive/Google_Rezensionen_qualitätsgesichert_2023_12_19 (1).xlsx'))
read_file.to_csv("Google_Rezensionen_qualitätsgesichert_2023_12_19 (1).csv", index = False, header = True, encoding=smartEncoding())
df = pd.DataFrame(pd.read_csv("Google_Rezensionen_qualitätsgesichert_2023_12_19 (1).csv"))
#print(df.head)

In [ ]:
#wähle relevante Spalten
gewaehlte_spalten = df[["Erfahrungsbericht des Nutzers" , "Zufallszahl"]]

#speichere sie in neue csv datei
gewaehlte_spalten.to_csv("/content/drive/My Drive/erfahrungen_gefiltert.csv", index = False)




# Datenbereinigung

In [ ]:
erfahrungen_gefiltert = pd.read_csv("/content/drive/My Drive/erfahrungen_gefiltert.csv")
# dropna() entfernt NaN-Reihen aus anderen Spalten
erfahrungen_gefiltert.dropna(subset=['Erfahrungsbericht des Nutzers'], inplace=True)
# leere strings entfernen
erfahrungen_gefiltert.drop_duplicates(inplace=True)
#speichern
erfahrungen_gefiltert.to_csv("/content/drive/My Drive/erfahrungen_clean.csv", index=False)
print(f"FINAL: {len(erfahrungen_gefiltert)} Reihen")


#speichere sie in neue csv datei
gewaehlte_spalten.to_csv("/content/drive/My Drive/erfahrungen_clean.csv", index = False)


#zufällig n = 100 Berichte für erfahrungen_final.csv auswählen
erfahrungen_clean = pd.read_csv("/content/drive/My Drive/erfahrungen_clean.csv")
# dropna() entfernt NaN-Reihen aus anderen Spalten
erfahrungen_clean.dropna(subset=['Erfahrungsbericht des Nutzers'], inplace=True)
# leere strings entfernen
erfahrungen_clean.drop_duplicates(inplace=True)
erfahrungen_clean.to_csv("/content/drive/My Drive/erfahrungen_clean.csv", index = False)

# INDEX-Nummern zufällig wählen
zufalls_index = random.sample(range(len(erfahrungen_clean)), k=100)
zufall_100 = erfahrungen_clean.iloc[zufalls_index]



#speichern (NUR 100!)
zufall_100.to_csv("/content/drive/My Drive/zufall_100_berichte.csv", index=False)

print(f"{len(zufall_100)} Zufallsberichte!")

FINAL: 6453 Reihen
100 Zufallsberichte!


# **Datenaugmentation und Syn-Chain-Methode**

## Datenaugmentation mit Trainingsdaten (ca. 11.900 Rezensionen)

## Paraphrasieren

In [ ]:
# === 3. Google Drive mounten (für persistente Speicherung) ===
drive.mount('/content/drive')
SAVE_PATH = '/content/drive/MyDrive/ABSA_Paraphrase/'
os.makedirs(SAVE_PATH, exist_ok=True)

# === 4. Gerät: GPU → CPU Fallback ===
use_gpu = torch.cuda.is_available()
device = 0 if use_gpu else -1
print(f"Gerät: {'GPU (CUDA)' if use_gpu else 'CPU'}")

if use_gpu:
    torch.backends.cudnn.benchmark = True
    print("CUDNN aktiviert")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Gerät: GPU (CUDA)
CUDNN aktiviert


In [ ]:

model_name = "google/flan-t5-large"  # 780M → beste Paraphrase-Qualität

try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    paraphraser = pipeline(
        "text2text-generation",
        model=model,
        tokenizer=tokenizer,
        device=device,
        max_length=512,
        batch_size=4 if use_gpu else 1
    )
    print(f"FLAN-T5-LARGE geladen auf {'GPU' if use_gpu else 'CPU'}")
except Exception as e:
    print(f"Fehler mit large → Fallback auf base")
    model_name = "google/flan-t5-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    paraphraser = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=device)

# === 6. Paraphrasieren mit Fallback ===
def paraphrase_review(text, num_variants=3, temperature=0.9):
    global paraphraser, device

    prompt = f"Paraphrase this review, keep aspect and sentiment: {text}"

    # GPU-Check
    if device == 0 and not torch.cuda.is_available():
        print("GPU NICHT MEHR VERFÜGBAR → Wechsel auf CPU")
        device = -1
        paraphraser = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=-1)

    try:
        results = paraphraser(
            prompt,
            max_length=512,
            num_return_sequences=num_variants,
            temperature=temperature,
            do_sample=True,
            top_p=0.95,
            truncation=True
        )
        return [r['generated_text'].strip() for r in results]
    except Exception as e:
        print(f"Fehler: {e} → Fallback auf Original")
        return [text] * num_variants
# === 7. CSV hochladen ===

df = pd.read_csv("/content/drive/MyDrive/erfahrungen_clean.csv")
print(f"{len(df)} Zeilen geladen.")
df.head()


Device set to use cuda:0


FLAN-T5-LARGE geladen auf GPU
6453 Zeilen geladen.


,Erfahrungsbericht des Nutzers,Zufallszahl
0,Ich arbeite seit ca. 10 Jahren im Botegunhof u...,2463
1,Seit drei Monaten arbeite ich als Praktikant i...,2511
2,Dieses Haus ist nicht zu empfehlen. Meine Oma ...,10419
3,Vielen Dank für die lieben Wünsche und Herzlic...,8718
4,Da freue ich mich immer auf eine lustige Gesel...,12483


In [1]:
# Importe


import torch
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
import time
import os
from google.colab import drive

# Google Drive mounten
drive.mount('/content/drive')
SAVE_PATH = '/content/drive/MyDrive/ABSA_Paraphrase/'
os.makedirs(SAVE_PATH, exist_ok=True)

# Gerät prüfen
use_gpu = torch.cuda.is_available()
device = 0 if use_gpu else -1
print(f"Gerät: {'GPU (CUDA)' if use_gpu else 'CPU'}")
if use_gpu:
    torch.backends.cudnn.benchmark = True
    print("CPU aktiviert")

# Modell laden (FLAN-T5-LARGE mit Fallback)
model_name = "google/flan-t5-large"

try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    paraphraser = pipeline(
        "text2text-generation",
        model=model,
        tokenizer=tokenizer,
        device=device,
        max_length=512,
        batch_size=4 if use_gpu else 1
    )
    print(f"FLAN-T5-LARGE geladen auf {'GPU' if use_gpu else 'CPU'}")
except Exception as e:
    print(f"Fehler mit large → Fallback auf base")
    model_name = "google/flan-t5-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    paraphraser = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=device)

# Paraphrasierungs-Funktion
def paraphrase_review(text, num_variants=3, temperature=0.9):
    global paraphraser, device
    prompt = f"Paraphrase this review, keep aspect and sentiment: {text}"

    if device == 0 and not torch.cuda.is_available():
        print("GPU weg → CPU-Fallback")
        device = -1
        paraphraser = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=-1)

    try:
        results = paraphraser(
            prompt,
            max_length=512,
            num_return_sequences=num_variants,
            temperature=temperature,
            do_sample=True,
            top_p=0.95,
            truncation=True
        )
        return [r['generated_text'].strip() for r in results]
    except Exception as e:
        print(f"Fehler: {e} → Fallback")
        return [text] * num_variants

# CSV laden
df = pd.read_csv("/content/drive/MyDrive/erfahrungen_clean.csv")
print(f"Originale: {len(df)} Zeilen")

# Checkpoint laden & Fortsetzen
augmented_data = []
checkpoint_file = SAVE_PATH + "reviews_paraphrased_checkpoint.csv"
final_file = SAVE_PATH + "reviews_paraphrased_final.csv"

start_idx = 0
if os.path.exists(checkpoint_file):
    checkpoint_df = pd.read_csv(checkpoint_file)
    augmented_data = checkpoint_df.to_dict('records')
    start_idx = len(augmented_data) // 4
    print(f"Checkpoint gefunden! Fortsetzen ab Original-Zeile {start_idx}")
else:
    print("Kein Checkpoint → Starte von vorne")

# Paraphrasieren (ab start_idx)
text_col = 'Erfahrungsbericht des Nutzers'
aspect = ''
sentiment = ''

for idx in tqdm(range(start_idx, len(df)), desc="Paraphrasiere"):
    row = df.iloc[idx]
    text = str(row[text_col]).strip()
    if text in ['nan', 'None', '']:
        text = "Leer"

    # Original
    augmented_data.append({'review_text': text, 'aspect': aspect, 'sentiment': sentiment})

    # Definition Paraphrasen
    variants = paraphrase_review(text, num_variants=3)
    for v in variants:
        augmented_data.append({'review_text': v.strip(), 'aspect': aspect, 'sentiment': sentiment})

    # Checkpoint alle 25 Originale
    if (idx + 1) % 25 == 0:
        pd.DataFrame(augmented_data).to_csv(checkpoint_file, index=False)
        print(f"Checkpoint: {len(augmented_data)} Zeilen (Original {idx+1}/{len(df)})")

    time.sleep(0.05)

# === 9. Finale Speicherung ===
df_final = pd.DataFrame(augmented_data)
df_final.to_csv(final_file, index=False)
df_final.to_csv(checkpoint_file, index=False)

print(f"\nFERTIG! {len(df_final)} Zeilen → {final_file}")
print(f"→ Erweitert um das {len(df_final)//len(df)}-fache!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Gerät: CPU


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Checkpoint: 17300 Zeilen (Original 4325/6453)


Paraphrasiere:  15%|█▌        | 375/2478 [6:19:22<13:35:08, 23.26s/it]

Checkpoint: 17400 Zeilen (Original 4350/6453)


Paraphrasiere:  16%|█▌        | 392/2478 [6:25:36<34:11:58, 59.02s/it]


KeyboardInterrupt: 

In [ ]:
# Download paraphrased file
files.download(final_file)
print("Auch in Google Drive gespeichert:", final_file)

für Fine Tuning:
Hugging Face: Fine-tune ein Grok-ähnliches Modell (z. B. "xai-org/grok-1" oder "reedmayhew/Grok-3-gemma3-4B-distilled" als Distillation von Grok 3). Das ist open-source und GitHub-freundlich.

Code-Beispiel (Distillation von Grok 3 via API + Fine-Tuning auf Gemma-3 4B):


Best Practices (aus Reddit, GitHub Blog, HF Docs):

Lizenz: Füge eine LICENSE (z. B. Apache 2.0) hinzu, da Grok-Modelle proprietär sind – dein Fine-Tuning erbt das.
Model Card: Erstelle eine (HF-Template: https://huggingface.co/docs/hub/model-cards) mit Details zu Daten (12.000 Rezensionen), Augmentation und Metriken.
Datenschutz: Anonymisiere sensible Daten in der CSV (z. B. Namen entfernen), bevor du hochlädst – Pflegeberichte könnten personenbezogen sein (DSGVO-konform!).
Versionskontrolle: Nutze GitHub Releases für große Dateien (z. B. Weights als Asset).
Integration mit HF: Push zu Hugging Face Hub und verlinke im GitHub-Repo – HF ist GitHub-ähnlich und unterstützt Grok-Destillationen (z. B. "reedmayhew/Grok-3-gemma3-4B-distilled").



3. Empfehlung für deine Bachelorarbeit

Starte mit Option A/B: Prompt Engineering für schnelle Tests, dann Hugging Face für echtes Fine-Tuning (ca. 30–60 Min. in Colab mit GPU).
GitHub-Repo als Portfolio: Lade Code, Skripte und Metriken hoch – das zeigt Reproduzierbarkeit. Vermeide rohe Weights (>100 MB) direkt; verlinke sie.
Nächste Schritte: Hole dir einen xAI API-Key und HF-Token. Teste den Code in Colab.

Falls du Hilfe beim Setup (z. B. API-Key, spezifischer Code-Fehler) brauchst oder mehr Details zu einer Option, lass es mich wissen!

In [ ]:
# um damit zu beginnen: https://arxiv.org/html/2507.09485